**Install Dependencies**

In [ ]:
# Install required packages
!pip install PyWavelets huggingface_hub scikit-image pandas tenacity numpy scipy

**Import and Setup**

In [ ]:
#CORE IMPORTS
import os
import tarfile
import io
import pandas as pd
import numpy as np
import shutil
import logging
from skimage import io as skio
from skimage import color, measure, feature
from skimage.util import random_noise, img_as_float
from scipy import ndimage as ndi
from skimage.restoration import estimate_sigma
from huggingface_hub import HfApi, hf_hub_download
from tenacity import retry, stop_after_attempt, wait_exponential

#GOOGLE DRIVE SETUP
from google.colab import drive
drive.mount('/content/drive')  # Connect to Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Configuration**

In [ ]:
#PATHS & SETTINGS
OUTPUT_DIR = "/content/drive/MyDrive/Deepfake"  # Where to save results
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Create folder if missing
IMAGE_SAVE_DIR = os.path.join(OUTPUT_DIR, "organized_images")
CATEGORIES = ['fake_train', 'fake_test', 'real_train', 'real_test']

#DATASET CONFIG
HF_TOKEN = "replace with your token here"  # Hugging Face access
DATASET_REPO = "xingjunm/WildDeepfake"             # Dataset name

#LOGGING SETUP
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
# Test folder creation
print(f"Main output will go to: {OUTPUT_DIR}")
print(f"Images will save to: {IMAGE_SAVE_DIR}")

# Verify category folders
for category in CATEGORIES:
    category_path = os.path.join(IMAGE_SAVE_DIR, category)
    os.makedirs(category_path, exist_ok=True)
    print(f"Created: {category_path}")

Main output will go to: /content/drive/MyDrive/Deepfake
Images will save to: /content/drive/MyDrive/Deepfake/organized_images
Created: /content/drive/MyDrive/Deepfake/organized_images/fake_train
Created: /content/drive/MyDrive/Deepfake/organized_images/fake_test
Created: /content/drive/MyDrive/Deepfake/organized_images/real_train
Created: /content/drive/MyDrive/Deepfake/organized_images/real_test


**Feature Extraction**

In [ ]:
def extract_features(img):
    """
    Extracts 6 key features from an image:
    1. Entropy - Measures image complexity
    2. Wrapped phase - Detects artificial patterns
    3. Noise level - Estimates digital manipulation
    4. Blur - Calculates focus consistency
    5. Keypoints - Counts unique visual features
    6. Blobs - Identifies irregular shapes
    """
    try:
        gray_img = color.rgb2gray(img)


        # Entropy
        entropy = measure.shannon_entropy(img)

        # Wrapped
        image_wrapped = np.angle(np.exp(1j * img))
        wrapped_range = np.max(image_wrapped) - np.min(image_wrapped)

        # Noise
        astro = img_as_float(img)
        astro = astro[30:180, 150:300]
        sigma = 0.08
        noisy = random_noise(astro, var=sigma**2)
        try:
            sigmas = estimate_sigma(noisy, channel_axis=-1)
        except TypeError:
            sigmas = estimate_sigma(noisy, multichannel=True)
        sigma_est = np.mean(sigmas)

        # Blur
        blurred_images = [ndi.uniform_filter(img, size=k) for k in range(2, 32, 2)]
        img_stack = np.stack(blurred_images)

        # Keypoints
        detector = feature.CENSURE()
        detector.detect(gray_img)

        # Blobs
        blobs_dog = feature.blob_dog(gray_img, max_sigma=1, threshold=0.1)


        return {
            'entropy': entropy,
            'wrapped': wrapped_range,
            'noise': sigma_est,
            'blur': np.mean(img_stack),
            'keypoints': len(detector.keypoints),
            'blobs': len(blobs_dog)
        }
    except Exception as e:
        logging.error(f"Feature extraction failed: {str(e)}")
        return None

**File Processing**

In [ ]:
# Modify the configuration
CATEGORIES = ['fake_train', 'fake_test', 'real_train', 'real_test']  # All categories
IMAGE_SAVE_DIR = os.path.join(OUTPUT_DIR, "organized_images")

# Create category subfolders
for category in CATEGORIES:
    os.makedirs(os.path.join(IMAGE_SAVE_DIR, category), exist_ok=True)

def process_tar_file(repo_id, file_path, label):
    entries = []
    try:
        local_path = hf_hub_download(...)

        with tarfile.open(local_path, "r:*") as tar:
            for member in tar.getmembers():
                if member.isfile() and member.name.lower().endswith('.png'):
                    try:
                        # Extract image
                        f = tar.extractfile(member)
                        img_data = f.read()

                        # ===== NEW: Organized Saving =====
                        category = 'fake_train' if 'fake_train' in file_path else \
                                  'fake_test' if 'fake_test' in file_path else \
                                  'real_train' if 'real_train' in file_path else 'real_test'

                        img_name = os.path.basename(member.name)
                        img_save_path = os.path.join(IMAGE_SAVE_DIR, category, img_name)

                        with open(img_save_path, 'wb') as img_file:
                            img_file.write(img_data)
                        # ===== END NEW =====

                        # Rest of processing...

                    except Exception as e:
                        logging.error(f"Error processing {member.name}: {str(e)}")
        os.remove(local_path)
    except Exception as e:
        logging.error(f"Failed to process {file_path}: {str(e)}")

    return entries

In [ ]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def process_tar_file(repo_id, file_path, label):
    """
    Processes a .tar.gz file with:
    - Automatic retries on failure
    - Image saving to Google Drive subfolders
    - Feature extraction
    - Clean temp file removal

    Returns: List of feature dictionaries
    """
    entries = []
    try:
        # Download file
        local_path = hf_hub_download(
            repo_id=repo_id,
            filename=file_path,
            token=HF_TOKEN,
            repo_type="dataset",
            cache_dir="tmp_downloads"
        )

        # Determine category (fake_train/fake_test/real_train/real_test)
        category = next((cat for cat in CATEGORIES if cat in file_path), None)
        if not category:
            raise ValueError(f"Could not determine category for {file_path}")

        # Process archive
        with tarfile.open(local_path, "r:*") as tar:
            for member in tar.getmembers():
                if member.isfile() and member.name.lower().endswith('.png'):
                    try:
                        # Extract image
                        f = tar.extractfile(member)
                        img_data = f.read()
                        img = skio.imread(io.BytesIO(img_data))

                        # ===== Save to Google Drive =====
                        save_dir = os.path.join(IMAGE_SAVE_DIR, category)
                        os.makedirs(save_dir, exist_ok=True)

                        # Clean filename: remove subfolder paths
                        img_name = os.path.basename(member.name)
                        save_path = os.path.join(save_dir, img_name)

                        with open(save_path, 'wb') as img_file:
                            img_file.write(img_data)
                        # ===== End saving =====

                        # Extract features
                        features = extract_features(img)

                        if features:
                            entries.append({
                                'Final_Entropy': features['entropy'],
                                'Final_Wrapped': features['wrapped'],
                                'Final_Noise': features['noise'],
                                'Final_Blur': features['blur'],
                                'Final_Keypoints': features['keypoints'],
                                'Final_Blobs': features['blobs'],
                                'Final_Label': label,
                                'Image_Path': save_path  # Google Drive path
                            })

                    except Exception as e:
                        logging.error(f"Error processing {member.name}: {str(e)}")

        # Cleanup
        os.remove(local_path)
        logging.info(f"Processed {len(entries)} images (saved to {category}/)")

    except Exception as e:
        logging.error(f"Failed to process {file_path}: {str(e)}")

    return entries

**Dataset Structure**

In [ ]:
def get_dataset_structure():
    """
    Gets the list of all .tar.gz files from Hugging Face,
    organized by category (fake_test only in this case)
    """
    api = HfApi(token=HF_TOKEN)
    files = api.list_repo_files(repo_id=DATASET_REPO, repo_type="dataset")

    file_map = {category: [] for category in CATEGORIES}

    for f in files:
        if f.endswith('.tar.gz'):
            for category in CATEGORIES:
                if f'deepfake_in_the_wild/{category}/' in f:
                    label = 'fake' if 'fake' in category else 'real'
                    file_map[category].append((f, label))
                    break

    return file_map

**Main Execution**

In [9]:
def main():
    """
    Orchestrates the entire workflow:
    1. Cleans temp files
    2. Gets file list
    3. Processes files in batches
    4. Saves progress checkpoints
    5. Outputs final CSV
    """
    if os.path.exists("tmp_downloads"):
        shutil.rmtree("tmp_downloads")

    # Get dataset structure
    file_map = get_dataset_structure()

    # Initialize DataFrame
    full_df = pd.DataFrame()

    # Process each category
    for category, files in file_map.items():
        logging.info(f"Processing {len(files)} files in {category}")

        for idx, (file_path, label) in enumerate(files):
            logging.info(f"Processing file {idx+1}/{len(files)}: {file_path}")
            entries = process_tar_file(DATASET_REPO, file_path, label)

            if entries:
                batch_df = pd.DataFrame(entries)
                full_df = pd.concat([full_df, batch_df], ignore_index=True)

                # Save incremental progress
                if (len(full_df) % 1000) == 0:
                    full_df.to_csv("processing_checkpoint.csv", index=False)

    # Final save
    final_path = os.path.join(OUTPUT_DIR, "final_processed_dataset.csv")
    full_df.to_csv(final_path, index=False)

    # Checkpoint save (if keeping)
    checkpoint_path = os.path.join(OUTPUT_DIR, "processing_checkpoint.csv")
    if (len(full_df) % 1000) == 0:
        full_df.to_csv(checkpoint_path, index=False)

if __name__ == "__main__":
    main()

1.tar.gz:   0%|          | 0.00/112M [00:00<?, ?B/s]

10.tar.gz:   0%|          | 0.00/8.95M [00:00<?, ?B/s]

100.tar.gz:   0%|          | 0.00/23.6M [00:00<?, ?B/s]

101.tar.gz:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

102.tar.gz:   0%|          | 0.00/95.7M [00:00<?, ?B/s]

...

244.tar.gz:   0%|          | 0.00/68.6M [00:00<?, ?B/s]

245.tar.gz:   0%|          | 0.00/28.9M [00:00<?, ?B/s]

246.tar.gz:   0%|          | 0.00/25.0M [00:00<?, ?B/s]

247.tar.gz:   0%|          | 0.00/71.1M [00:00<?, ?B/s]

KeyboardInterrupt: 